In [191]:
import pandas as pd
import re
SalesHeader = pd.read_csv('SalesHeader.csv')
SalesItem = pd.read_csv('SalesItem.csv')
Location = pd.read_csv('Location.csv')
Customer = pd.read_csv('Customer.csv')

### PROBLEM NUMBER 1:

In [ ]:
SQL QUERY, where:
si: SalesItem
cs: Customer
sh: SalesHeader
lc: Location

#1) create our target variable "TotalAmtPaid" and select to display both customer name and total amount paid as those
# are the final two columns we are interested in:
select cs."CustomerName", sum((si."AmtPaidPerCase"*si."Quantity")) as TotalAmtPaid  from customers cs

#2) proceed to join all data frames in consecutive order
   left join salesheader sh on cs."CustomerID" = sh."CustomerID"

left join items si on si."OrderID"=sh."OrderID"

left join location lc on lc."LocationID" = sh."ShipToID"

#3) introduce the limitations on the type of data we are interested in:
where lc."State"='TX' and lc."LocationType" = 'Store'
  
    and sh."OrderDate" >= '2018-09-01' AND sh."OrderDate" < '2018-10-01'
#4) finally groupby customer name to aggregate all individual amounts paid per customer and limit the resulting dataframe
#to a length of 10 rows.
group by cs."CustomerName"
limit 10;

### PROBLEM NUMBER 2:

In [247]:
#1Find out how much an order cost in total
SalesItem['TotalAmtPaid'] = SalesItem.Quantity * SalesItem.AmtPaidPerCase

#2 Merge data frame with date ordered, total cost and customer
ItemHead = pd.merge(SalesItem,SalesHeader)
ItemHeadCust = pd.merge(ItemHead,Customer)

#3 Subset main dataframe for Sept 2018 using regex:
IteamHeadCust = ItemHeadCust[ItemHeadCust.OrderDate.str.contains('09/[0-9]+/2018')]

#4 Merge location data frame with other 3 merged dataframes on ShipToID as we only want stores:
Alldata = pd.merge(ItemHeadCust, Location, left_on=  ['ShipToID'],
                   right_on= ['LocationID'], 
                   how = 'left')

#5 Subset location data for stores and TX only:
Alldata = Alldata[(Alldata.LocationType == 'Store') & (Alldata.State == 'TX') ]

#6 groupby customer name and sum total amount paid:
Final = Alldata.groupby('CustomerName').agg(sum).reset_index()

#6 Output top 10 customers with the highest total amount paid:
Final[['TotalAmtPaid','CustomerName']].sort_values(by = 'TotalAmtPaid',ascending = False)[:10]

,TotalAmtPaid,CustomerName
0,68.92,Fun Superstore


### PROBLEM NUMBER 3:

In [248]:
def add0_remove_sub5(df):
    #1Copy dataframe
    df1 = df.copy()

    #Convert all code IDs to strings:
    df1['OrderID'] = [str(x) for x in df1['OrderID']]
    
    #Eliminate all below 5:
    df1 = df1[df1['OrderID'].apply(lambda x: len(x)>4)]
    
    #Add 0s:
    df1['OrderID'] = [(8-len(x))*'0'+x for x in df1['OrderID']]
    return df1

add0_remove_sub5(SalesHeader)

,OrderID,OrderDate,CustomerID,ShipFromID,ShipToID
0,00152369,01/01/2018,159,174747,4236
1,86592586,09/04/2018,74,45944,775553
2,00075396,01/01/2018,2583,45944,253447
3,65324852,07/28/2018,9,145823,857456
4,00074474,08/14/2018,4564,685144,857456
5,00159447,01/03/2018,853,915354,954835
6,00654151,11/15/2018,2583,760613,15501
7,02315454,09/04/2018,2583,233054,86592586
9,14881448,05/17/2018,6895,468871,929366


### PROBLEM NUMBER 4:
I would create a multiple linear regression model to predict the AmtPaidPerCase under the assumption that there is a relationship between the item ID, quantity ordered, distance between to and from locations and the type of customer. If R^2>0.7, I would use this model to predict the amount paid per case. The reason why I would use this method instead of random or mean imputation is because it seems to me that the amount paid per case should logically vary based on other features in the dataset. <br>
Hot deck imputation could also be compared with regression based imputaion since it randomly choes a values from a sample that has similar values for other features. This is essentially a clustering method, which might actually yield better results depending on the relationships between the features and the amount paid per case.

### PROBLEM NUMBER 5:
#### 1) data/feature engineering: (python: pandas, reg.ex, numpy and seaborn)
- Change the date into three different column features: day, month and year so as to accurately capture the relationship with time.<br>
- Merge all dataframes so that we have 1 dataframe with all the information. Make sure to merge the location dataframe onto both shipIDfrom and shipIDto.<br>
- Delete redundant information such as OrderID, customerID, locationID and ship to/from IDs since now our dataframes are merged.<br>
- Create a couple of plots to better understand the data: distribution of total amount of items sold per day, matrix of lin. reg. plots between price and each feature and matrix of lin. reg. plots between features to identify auto-correlation<br>
- Look at summary statistics in groupby dataframes, histograms and boxplots to identify outliers.<br>
- If there is high auto-correlation, use VIF = 4 as a threshold to eliminate features.<br>
- No dummifying needs to be done for ordinal or categorical variables.<br>

#### 2) modeling: (python: sklearn (ensemble & model_selection))
- Type of problem: supervised learning regression problem
- Remarks about model types:<br>
    - We don't have many features, so no issues with curse of dimensionality.<br>
    - Deep learning models don't seem too useful here as the data-set is simplistic and predicting demand is extremely hard to pinpoint in retail even with ML/AI. <br>
    - I don't think we necessarily have a linear relationship between features and predicted paid amount, so a model that can accomodate non-linear relationships might be good.<br>
    - Might want a model that allows for feature interaction to better capture the relationships which might not necessarily be monotone.<br>
    - Considering the type of data and the end goal of having the model be utilized by the business, it might be nice to have a certain amount of interpretability.<br>
- I think a random forest model would fulfill the above confiditions.<br>
- Would use the random forest regresion from sklearn with the randomized cross-validation search to accurately score the parameters chosen for that model. Depending on how many data points we have, we could have anywhere from 2 folds, to 100+.

### PROBLEM NUMBER 6:

Similar to the last part of my answer to problem 6, to best evaluate the model, I would use cross-validation (train test splt method) to ensure my model scoring is accurate.
The number of folds would be determined by the amount of data avaialble where the more data points I have, the more folds I can afford to make and not run the risk of overfitting. I would use both an adjusted R^2 value as well as the RMSE to score my models and fine tune my tree's hyperparameters. R^2 measures the amount of variance in the target variable that is  explained by the model, whereas RMSE is the amount of error in the units of the target variabele. RMSE more simply put is the standard deviation of the unexplained variance. The combination of these provides a good overall scoring methodology that would be outputted during the grid CV hyperperameter search.

### PROBLEM NUMBER 7:
Automating our dashboard comes down to streamlining the ETL process a.k.a. extracting, transforming and loading our data.<br>
1) Extract the data, whether that be from a data lake, SQL (a relational database) or a data warehouse, I would use Spark to first access that data, extract it and partition if for processing.<br>
2) Once the data has been extracted from it's main warehousing source I would have a series of dataframe transforming functions that would tackle the pulled data. Spark allows this to be done very quickly as it is a distributed engine (runs computations in parallel) and allows for the chaining of transforming functions.<br>
3) As Spark is processing the data, I would write it into S3 (Amazon's Simple Storage Service would have be be leased) without having to setup a server. Databrick's platform as well as its API are fantastic for building out this Spark pipeline, which would also involve a financial investment.<br>
4) At this point our data is ready to be fed into a BI tool, whether that be a model or a vizualization. A data inflow would trigger the events of steps 1 through 4 all over again. I would setup this trigger by having a set of conditions that need to be fulfilled before commencing the process.


